In [1]:
import plot_functions as pf
import pandas as pd
import numpy as np
from   scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import time
import importlib

In [16]:
class DegradationModel:
    """
    Implementation of Colin et al. (2009) PE pipe degradation model
    with chlorine dioxide (DOC) exposure, focusing on diffusion-reaction coupling.
    Can be adapted for film simulations by changing boundary conditions.

    References:
    [1] Colin, X., et al. (2009). Polymer Engineering & Science, 49(7), 1429-1437. (Part I)
    [2] Colin, X., et al. (2009). Polymer Engineering & Science, 49(8), 1642-1652. (Part II)
    [3] Colin, X., et al. (2009). Macromolecular Symposia, 286(1), 81-88.
    """

    def __init__(self, L=4.5e-3, nz=100, Mw0=150, dens0=0.95, densa=0.85, Xc=0.45, simulation_mode='pipe'):
        """
        Initialize model parameters, grid, and species indices.

        Args:
            L (float): Pipe wall thickness (m) or Film thickness (m).
            nz (int): Number of grid points for spatial discretization.
            Mw0 (float): Initial molecular weight of PE (Kg/mol).
            dens0 (float): Density of PE (Kg/L).
            simulation_mode (str): 'pipe' or 'film'. Determines boundary conditions.
        """
        self.L = L
        self.nz = nz
        self.z = np.linspace(0, L, nz) # Grid points (m)
        self.dz = L / (nz - 1)         # Grid spacing (m)
        self.Mw0 = Mw0                 # Convert to g/mol (need to check units in the paper)
        self.dens0 = dens0             # Densité totale du PE (kg/L)
        self.Xc = Xc           # Fraction cristalline (adimensionnelle)
        self.densa = densa     # Densité de la phase amorphe (kg/L)
        self.Tam = 1.0 - self.Xc # Fraction amorphe (volumique ou massique, à clarifier)
        
        # Calcul de Tav, si vous l'utilisez explicitement pour modifier les taux de réaction
        # Assurez-vous que dens0 et densa sont dans les mêmes unités (ex: kg/L)
        if self.densa > 1e-9: # Eviter division par zéro
            self.Tav = (self.dens0 / self.densa) * self.Tam
        else:
            self.Tav = self.Tam # Approximation si densa n'est pas fournie ou nulle                

        if simulation_mode not in ['pipe', 'film']:
            raise ValueError("simulation_mode must be 'pipe' or 'film'")
        self.simulation_mode = simulation_mode
        print(f"Model initialized in '{self.simulation_mode}' mode with L={self.L:.2e}m, nz={self.nz}.")

        # --- Mechanical Parameters (Level 3) ---
        self.A0 = -29.7
        self.H_plus_m_term = 166725.0
        self.alpha_m = 3.2
        self.stress_exp_b = 3.3
        self.critical_depth_m = 100e-6
        self.MF_crit = 70.0

        # --- Model Parameters ---
        self.R_gas = 8.32 # J / (mol K)
        # Paramètres pour la solubilité du DOC (inspirés du Matlab, à vérifier/affiner)
        self.S0_DOC = 2.6e-9  # mol/L_amorphe_PE / Pa (Pré-exponentiel solubilité DOC)
        self.ES_DOC = -2690   # K (Terme -Hs_DOC/R pour solubilité DOC)
        self.pd0_DOC = 5.7e4  # Pa/ppm (Pré-exponentiel conversion ppm_eau en P_partielle_DOC)
        self.Ep_DOC = 26440   # J/mol (Terme -E_vap_eff/R pour pression partielle DOC)
                            # Note: Matlab avait Ep/R en K, donc ici Ep est J/mol

        # --- Rate Coefficients (k_coeffs) ---
        self.k_coeffs = {
            'k1d': (0, 2.7e-5), 'k1u': (140e3, 8.0e12), 'k1b': (105e3, 2.8e9),
            'k2': (0, 1.0e8), 'k3': (73e3, 1.5e10), 'k4':  (0, 8.0e11),   
            'k4d':  (21.1e3, 6.6e9), 'k5':  (5.9e3, 1.5e12), 'k60':  (80e3, 4.9e19), 
            'k61': (0, 2e6), 'k62': (5e3, 1.2e6), 'k63': (17.4e3, 4.8e9),
            'k7': (49.9e3, 1.3e9), 'k8d':  (0, 5.0e-2),
        }

        # --- Diffusion Coefficients (D_coeffs) at 15°C ---
        self.D_coeffs = {
            'O2':   (35e3, 4.3e-5), 'DOC':  (0, 2.0e-11), 'AH':   (115.7e3, 9.1e4),
        }

        # --- Boundary Coefficients (beta_coeffs) at 15°C ---
        self.beta_coeffs = {
            'beta0': (0, 1.9e-9), # Extraction (z=0, water)
            'betaL': (0, 1.0e-10),# Evaporation (z=L, air - for pipe mode)
        }
        
        if self.simulation_mode == 'film':
            # For an immersed film, physical loss at z=L is by water extraction,
            # similar to z=0. We make betaL use beta0's parameters.
            beta0_Ea, beta0_A = self.beta_coeffs['beta0']
            self.beta_coeffs['betaL'] = (beta0_Ea, beta0_A) 
            print(f"  Film mode: beta_coeffs['betaL'] parameters set to beta_coeffs['beta0'] (Ea={beta0_Ea}, A={beta0_A})")


        # --- Yields (gamma) ---
        self.gamma = {
            'y1s': 1.0, 'y1co': 0.61, 'y4': 0.5, 'y5': 0.0,
        }

        # --- Stoichiometric / Other Parameters ---
        self.n_AH = 4      
        self.PH0_conc = 60.0 
        self.POOH0_conc = 1.0e-2 
        self.O2_sat_conc = 3.8e-4 
        self.AH0_conc = 1.8e-3 # Default initial AH conc for model base material
        self.DOC_conversion_factor = 1.7e-5 
        self.ti0_oit = 165.0 # Default initial OIT, can be overridden for specific materials

        # --- Species Indices ---
        self.species = ['O2', 'DOC', 'AH', 'P', 'PO2', 'POOH', 'PH', 'Q', 'CO', 'PCl', 'S', 'X']
        self.n_species = len(self.species)
        self.idx = {name: i for i, name in enumerate(self.species)}

        self.current_T_K = None
        self.k = {}
        self.D = {}
        self.beta = {}
        self.C0 = None # Will be set in simulate

    def _update_params_for_temp(self, T_kelvin):
        if T_kelvin == self.current_T_K: # Avoid redundant calculations
            return

        self.current_T_K = T_kelvin
        for name, (Ea, A) in self.k_coeffs.items():
            self.k[name] = A * np.exp(-Ea / (self.R_gas * T_kelvin)) if Ea > 0 else A
        for name, (Ea, D_val) in self.D_coeffs.items(): # Renamed D to D_val to avoid conflict
            self.D[name] = D_val * np.exp(-Ea / (self.R_gas * T_kelvin)) if Ea > 0 else D_val
        for name, (Ea, beta_val) in self.beta_coeffs.items(): # Renamed beta to beta_val
             self.beta[name] = beta_val * np.exp(-Ea / (self.R_gas * T_kelvin)) if Ea > 0 else beta_val
        # Calcul de la solubilité du DOC et du facteur de pression partielle à T_kelvin
        self.Sd_DOC_T = self.S0_DOC * np.exp(-self.ES_DOC / T_kelvin) # mol/L_amorphe_PE / Pa
        self.pd_DOC_T = self.pd0_DOC * np.exp(-self.Ep_DOC / (self.R_gas * T_kelvin)) # Pa/ppm_eau

    def system_equations(self, t, y):
        nz = self.nz
        n_species = self.n_species
        dz = self.dz
        idx = self.idx
        k = self.k
        D = self.D
        beta = self.beta
        gamma = self.gamma
        n_AH = self.n_AH

        C = y.reshape((n_species, nz))
        dCdt = np.zeros_like(C)

        O2   = C[idx['O2']]
        DOC  = C[idx['DOC']]
        AH   = C[idx['AH']]
        P    = C[idx['P']]
        PO2  = C[idx['PO2']]
        POOH = C[idx['POOH']]
        PH   = C[idx['PH']]
        Q    = C[idx['Q']]

        # --- Interior points ---
        for i in range(1, nz - 1):
            dCdt[idx['O2'], i] = D['O2'] * (O2[i+1] - 2*O2[i] + O2[i-1]) / (dz**2) \
                                - k['k2'] * O2[i] * P[i] + k['k60'] * PO2[i]**2
            dCdt[idx['DOC'], i] = D['DOC'] * (DOC[i+1] - 2*DOC[i] + DOC[i-1]) / (dz**2) \
                                - k['k1d'] * DOC[i] * PH[i] \
                                - k['k4d'] * P[i] * DOC[i] \
                                - n_AH * k['k8d'] * DOC[i] * AH[i]
            dCdt[idx['AH'], i] = D['AH'] * (AH[i+1] - 2*AH[i] + AH[i-1]) / (dz**2) \
                                - n_AH * k['k8d'] * DOC[i] * AH[i] \
                                - n_AH * k['k7'] * PO2[i] * AH[i]
            dCdt[idx['P'], i] = k['k1d'] * DOC[i] * PH[i] + 2 * k['k1u'] * POOH[i] \
                            + k['k1b'] * POOH[i]**2 - k['k2'] * O2[i] * P[i] \
                            + k['k3'] * PH[i] * PO2[i] - 2 * k['k4'] * P[i]**2 \
                            - k['k4d'] * P[i] * DOC[i] - k['k5'] * P[i] * PO2[i] \
                            + 2 * k['k63'] * Q[i]
            dCdt[idx['PO2'], i] = k['k1b'] * POOH[i]**2 + k['k2'] * O2[i] * P[i] \
                                - k['k3'] * PH[i] * PO2[i] - k['k5'] * P[i] * PO2[i] \
                                - 2 * k['k60'] * PO2[i]**2 - n_AH * k['k7'] * PO2[i] * AH[i]
            dCdt[idx['POOH'], i] = -k['k1u'] * POOH[i] - 2 * k['k1b'] * POOH[i]**2 \
                                + k['k3'] * PH[i] * PO2[i] \
                                + (1 - gamma['y5']) * k['k5'] * P[i] * PO2[i]
            dCdt[idx['Q'], i] = k['k60'] * PO2[i]**2 - (k['k61'] + k['k62'] + k['k63']) * Q[i]
            dCdt[idx['PH'], i] = -k['k1d'] * DOC[i] * PH[i] \
                                - (2 + gamma['y1s']) * k['k1u'] * POOH[i] \
                                - (1 + gamma['y1s']) * k['k1b'] * POOH[i]**2 \
                                - k['k3'] * PH[i] * PO2[i] \
                                + 2 * gamma['y4'] * k['k4'] * P[i]**2 \
                                + (3 * gamma['y5'] - 1) * k['k5'] * P[i] * PO2[i] \
                                + 2 * k['k61'] * Q[i] - 2 * (1 + gamma['y1s']) * k['k63'] * Q[i]
            dCdt[idx['CO'], i] = gamma['y1co'] * k['k1u'] * POOH[i] \
                                + gamma['y1co'] * k['k1b'] * POOH[i]**2 \
                                + k['k62'] * Q[i] + 2 * gamma['y1co'] * k['k63'] * Q[i]
            dCdt[idx['PCl'], i] = k['k4d'] * P[i] * DOC[i]
            dCdt[idx['S'], i] = gamma['y1s'] * k['k1u'] * POOH[i] \
                                + gamma['y1s'] * k['k1b'] * POOH[i]**2 \
                                + 2 * gamma['y1s'] * k['k63'] * Q[i]
            dCdt[idx['X'], i] = gamma['y4'] * k['k4'] * P[i]**2 \
                                + gamma['y5'] * k['k5'] * P[i] * PO2[i] + k['k61'] * Q[i]

        # --- Boundary Conditions ---
        # === z = 0 (Inner surface, always water interface) ===
        dCdt[idx['O2'], 0] = 0 
        dCdt[idx['DOC'], 0] = 0 
        dCdt[idx['AH'], 0] = -beta['beta0'] * AH[0] \
                             - n_AH * k['k8d'] * DOC[0] * AH[0] \
                             - n_AH * k['k7'] * PO2[0] * AH[0]
        # Immobile species at z=0 (Reaction only)
        dCdt[idx['P'], 0] = k['k1d'] * DOC[0] * PH[0] + 2 * k['k1u'] * POOH[0] + k['k1b'] * POOH[0]**2 - k['k2'] * O2[0] * P[0] + k['k3'] * PH[0] * PO2[0] - 2 * k['k4'] * P[0]**2 - k['k4d'] * P[0] * DOC[0] - k['k5'] * P[0] * PO2[0] + 2 * k['k63'] * Q[0]
        dCdt[idx['PO2'], 0] = k['k1b'] * POOH[0]**2 + k['k2'] * O2[0] * P[0] - k['k3'] * PH[0] * PO2[0] - k['k5'] * P[0] * PO2[0] - 2 * k['k60'] * PO2[0]**2 - n_AH * k['k7'] * PO2[0] * AH[0]
        dCdt[idx['POOH'], 0] = - k['k1u'] * POOH[0] - 2 * k['k1b'] * POOH[0]**2 + k['k3'] * PH[0] * PO2[0] + (1 - gamma['y5']) * k['k5'] * P[0] * PO2[0]
        dCdt[idx['Q'], 0] = k['k60'] * PO2[0]**2 - (k['k61'] + k['k62'] + k['k63']) * Q[0]
        dCdt[idx['PH'], 0] = - k['k1d'] * DOC[0] * PH[0] - (2 + gamma['y1s']) * k['k1u'] * POOH[0] - (1 + gamma['y1s']) * k['k1b'] * POOH[0]**2 - k['k3'] * PH[0] * PO2[0] + 2 * gamma['y4'] * k['k4'] * P[0]**2 + (3 * gamma['y5'] - 1) * k['k5'] * P[0] * PO2[0] + 2 * k['k61'] * Q[0] - 2 * (1 + gamma['y1s']) * k['k63'] * Q[0]
        dCdt[idx['CO'], 0] = gamma['y1co'] * k['k1u'] * POOH[0] + gamma['y1co'] * k['k1b'] * POOH[0]**2 + k['k62'] * Q[0] + 2 * gamma['y1co'] * k['k63'] * Q[0]
        dCdt[idx['PCl'], 0] = k['k4d'] * P[0] * DOC[0]
        dCdt[idx['S'], 0] = gamma['y1s'] * k['k1u'] * POOH[0] + gamma['y1s'] * k['k1b'] * POOH[0]**2 + 2 * gamma['y1s'] * k['k63'] * Q[0]
        dCdt[idx['X'], 0] = gamma['y4'] * k['k4'] * P[0]**2 + gamma['y5'] * k['k5'] * P[0] * PO2[0] + k['k61'] * Q[0]
        
        # === z = L (Outer surface) ===
        dCdt[idx['O2'], -1] = 0 # Fixed C[O2,-1]

        if self.simulation_mode == 'film':
            dCdt[idx['DOC'], -1] = 0 # Dirichlet for DOC (value was set in C0)
            # AH at z=L for film: extraction by water (beta['betaL'] was made equal to beta['beta0'] effectively)
            dCdt[idx['AH'], -1] = -beta['betaL'] * AH[-1] \
                                  - n_AH * k['k8d'] * DOC[-1] * AH[-1] \
                                  - n_AH * k['k7'] * PO2[-1] * AH[-1]
        elif self.simulation_mode == 'pipe':
            # DOC at z=L for pipe: No flux (Neumann) + Reactions
            dCdt[idx['DOC'], -1] = D['DOC'] * 2.0 * (DOC[-2] - DOC[-1]) / (dz**2) \
                                   - k['k1d'] * DOC[-1] * PH[-1] \
                                   - k['k4d'] * P[-1] * DOC[-1] \
                                   - n_AH * k['k8d'] * DOC[-1] * AH[-1]
            # AH at z=L for pipe: Evaporation (using original betaL definition)
            dCdt[idx['AH'], -1] = -beta['betaL'] * AH[-1] \
                                  - n_AH * k['k8d'] * DOC[-1] * AH[-1] \
                                  - n_AH * k['k7'] * PO2[-1] * AH[-1]
        
        # Immobile species at z=L (Reaction only)
        dCdt[idx['P'], -1] = k['k1d'] * DOC[-1] * PH[-1] + 2 * k['k1u'] * POOH[-1] + k['k1b'] * POOH[-1]**2 - k['k2'] * O2[-1] * P[-1] + k['k3'] * PH[-1] * PO2[-1] - 2 * k['k4'] * P[-1]**2 - k['k4d'] * P[-1] * DOC[-1] - k['k5'] * P[-1] * PO2[-1] + 2 * k['k63'] * Q[-1]
        dCdt[idx['PO2'], -1] = k['k1b'] * POOH[-1]**2 + k['k2'] * O2[-1] * P[-1] - k['k3'] * PH[-1] * PO2[-1] - k['k5'] * P[-1] * PO2[-1] - 2 * k['k60'] * PO2[-1]**2 - n_AH * k['k7'] * PO2[-1] * AH[-1]
        dCdt[idx['POOH'], -1] = -k['k1u'] * POOH[-1] - 2 * k['k1b'] * POOH[-1]**2 + k['k3'] * PH[-1] * PO2[-1] + (1 - gamma['y5']) * k['k5'] * P[-1] * PO2[-1]
        dCdt[idx['Q'], -1] = k['k60'] * PO2[-1]**2 - (k['k61'] + k['k62'] + k['k63']) * Q[-1]
        dCdt[idx['PH'], -1] = -k['k1d'] * DOC[-1] * PH[-1] - (2 + gamma['y1s']) * k['k1u'] * POOH[-1] - (1 + gamma['y1s']) * k['k1b'] * POOH[-1]**2 - k['k3'] * PH[-1] * PO2[-1] + 2 * gamma['y4'] * k['k4'] * P[-1]**2 + (3 * gamma['y5'] - 1) * k['k5'] * P[-1] * PO2[-1] + 2 * k['k61'] * Q[-1] - 2 * (1 + gamma['y1s']) * k['k63'] * Q[-1]
        dCdt[idx['CO'], -1] = gamma['y1co'] * k['k1u'] * POOH[-1] + gamma['y1co'] * k['k1b'] * POOH[-1]**2 + k['k62'] * Q[-1] + 2 * gamma['y1co'] * k['k63'] * Q[-1]
        dCdt[idx['PCl'], -1] = k['k4d'] * P[-1] * DOC[-1]
        dCdt[idx['S'], -1] = gamma['y1s'] * k['k1u'] * POOH[-1] + gamma['y1s'] * k['k1b'] * POOH[-1]**2 + 2 * gamma['y1s'] * k['k63'] * Q[-1]
        dCdt[idx['X'], -1] = gamma['y4'] * k['k4'] * P[-1]**2 + gamma['y5'] * k['k5'] * P[-1] * PO2[-1] + k['k61'] * Q[-1]
        
        return dCdt.flatten()

## This fuction runs the simulation for given conditions
    def simulate(self, T_celsius, DOC_ppm, t_end_years, n_timepoints=100, 
                 O2_sat_mult=1.0, AH0_mult=1.0, POOH0_mult=1.0, DOC_mult=1.0,
                 method='Radau', rtol=1e-4, atol=1e-7):
        
        T_kelvin = T_celsius + 273.15
        self._update_params_for_temp(T_kelvin) 

        O2_boundary_conc = self.O2_sat_conc * O2_sat_mult
        AH0_actual_conc = self.AH0_conc * AH0_mult
        POOH0_actual_conc = self.POOH0_conc * POOH0_mult
        # DOC_boundary_conc = DOC_ppm * self.DOC_conversion_factor * DOC_mult if DOC_ppm > 0 else 0.0 # ANCIENNE LIGNE
        if DOC_ppm > 0:
            # Concentration d'équilibre du DOC dans la phase amorphe du PE (mol/L_amorphe_PE)
            doc_conc_amorphous_phase = self.Sd_DOC_T * self.pd_DOC_T * DOC_ppm * DOC_mult
            
            # Si vos équations et constantes k sont pour des concentrations par volume TOTAL de PE:
            # On convertit la concentration dans la phase amorphe en concentration par volume total de PE
            # en multipliant par la fraction volumique amorphe accessible (Tam ou Tav).
            # Utilisons Tam = (1-Xc) comme approximation de la fraction volumique amorphe.
            DOC_boundary_conc = doc_conc_amorphous_phase * self.Tam 
            # Alternativement, si les 'k' sont pour des réactions en phase amorphe avec des conc. amorphes,
            # alors DOC_boundary_conc pourrait être doc_conc_amorphous_phase,
            # et les termes de réaction impliquant PH devraient utiliser PH_amorphe = PH0_conc / Tam.
            # Pour l'instant, cette approche rend DOC_boundary_conc équivalent à [mol_DOC / L_PE_total].
        else:
            DOC_boundary_conc = 0.0

        y0_flat = np.zeros(self.n_species * self.nz) # Keep as flat for direct use by solver
        C0 = y0_flat.reshape((self.n_species, self.nz)) # Temporary view for easy C0 setup

        C0[self.idx['O2'], :]   = O2_boundary_conc # Uniform O2 initially, including boundaries
        C0[self.idx['DOC'], :]  = 0.0              # DOC starts at 0 in bulk
        C0[self.idx['AH'], :]   = AH0_actual_conc 
        C0[self.idx['POOH'], :] = POOH0_actual_conc 
        C0[self.idx['PH'], :]   = self.PH0_conc   
        C0[self.idx['P'], :]    = 0.0  
        C0[self.idx['PO2'], :]  = 0.0  
        C0[self.idx['Q'], :]    = 0.0  
        C0[self.idx['CO'], :]   = 0.0  
        C0[self.idx['PCl'], :]  = 0.0  
        C0[self.idx['S'], :]    = 0.0  
        C0[self.idx['X'], :]    = 0.0  

        # Set specific boundary concentrations in C0 for Dirichlet conditions
        C0[self.idx['O2'], 0]   = O2_boundary_conc
        C0[self.idx['O2'], -1]  = O2_boundary_conc
        C0[self.idx['DOC'], 0]  = DOC_boundary_conc
        C0[self.idx['AH'], -1]   = AH0_actual_conc*(1-0.25) # OSL [AH]
        
        if self.simulation_mode == 'film':
            C0[self.idx['DOC'], -1] = DOC_boundary_conc 
            print(f"  FILM MODE: Initial C0[DOC, L] set to {DOC_boundary_conc:.2e}")
        # For pipe mode, C0[DOC, -1] remains the bulk initial value (0.0)
        # as its BC in system_equations is Neumann+Reaction

        self.C0 = C0.copy() # Store the fully prepared C0 if needed for reference
        y0 = C0.flatten() # Use the already flattened y0_flat which C0 was a view of

        t_start_sec = 0
        t_end_sec = t_end_years * 365.25 * 24 * 3600
        t_span = (t_start_sec, t_end_sec)
        # Ensure t_eval has at least 2 points if t_end_sec > t_start_sec for solve_ivp
        if np.isclose(t_start_sec, t_end_sec):
            actual_n_timepoints = 1 if n_timepoints >=1 else n_timepoints # Can be 1 for t=0
        else:
            actual_n_timepoints = max(2, n_timepoints)
        t_eval = np.linspace(t_start_sec, t_end_sec, actual_n_timepoints)


        print(f"Running simulation: T={T_celsius}°C, DOC={DOC_ppm} ppm, Time={t_end_years:.3f} years, Mode='{self.simulation_mode}'")
        print(f"  Solver: {method}, rtol={rtol:.1e}, atol={atol:.1e}")
        start_time = time.time()
        
        sol = solve_ivp(
            self.system_equations,
            t_span,
            y0, # Pass the fully prepared y0
            method=method, 
            t_eval=t_eval,
            rtol=rtol,  
            atol=atol   
        )
        end_time = time.time()
        print(f"Simulation duration: {end_time - start_time:.2f} seconds")

        if not sol.success:
            print(f"Simulation FAILED: {sol.message}")
        else:
            print("Simulation successful.")
            sol.sim_params = {
                'T_celsius': T_celsius, 'DOC_ppm': DOC_ppm, 't_end_years': t_end_years,
                'AH0_conc_used': AH0_actual_conc, 'POOH0_conc_used': POOH0_actual_conc,
                'O2_sat_conc_used': O2_boundary_conc, 'simulation_mode': self.simulation_mode,
                'rtol_used': rtol, 'atol_used': atol, 'method_used': method
            }
        return sol

## This function calculates the Oxidation Induction Time (OIT) based on the antioxidant profile
## AVEC CORRECTION 
    def calculate_oit(self, AH_profile, AH0_actual_conc):
        """
        Calculate Oxidation Induction Time (OIT) from AH profile.

        Args:
            AH_profile (numpy.ndarray): Array of current antioxidant (AH) concentrations
                                        at different spatial points (or over time at one point).
            AH0_actual_conc (float): The initial antioxidant (AH) concentration that
                                     AH_profile should be compared against for this specific
                                     simulation run or material.

        Returns:
            numpy.ndarray: Array of OIT values (min) corresponding to AH_profile.
        """
        # Using Eq (1) from Part II [2], relative to the *initial* AH concentration used
        # OIT(t) = ti0 * ([AH](t) / [AH]0)
        # self.ti0_oit should be set appropriately for the material being simulated
        # (e.g., 165 min for Sample A in Paper [2])

        # Protect against division by zero if AH0_actual_conc is zero or negative
        if AH0_actual_conc <= 1e-12: # Using a small threshold instead of direct zero check
            # If initial AH is effectively zero, OIT is also zero
            return np.zeros_like(AH_profile) 

        # Calculate the ratio, ensuring AH_profile values are not negative
        # (though they shouldn't be if the ODEs are well-behaved)
        ah_ratio = np.maximum(0 , AH_profile) / AH0_actual_conc
        
        oit = self.ti0_oit * ah_ratio
        
        # OIT cannot be negative, and practically, should not exceed initial OIT significantly
        # if AH_profile cannot exceed AH0_actual_conc.
        # Clamping OIT values to be non-negative.
        #oit = np.maximum(0, oit)
        
        # Optional: If you want to strictly ensure OIT doesn't exceed ti0_oit
        # (e.g., if AH_profile due to some numerical artifact slightly exceeds AH0_actual_conc)
        # oit = np.minimum(oit, self.ti0_oit) # This would cap OIT at its initial value

        return oit
## SANS CORRECTION
    def calculate_oit_sans_correction(self, AH_profile, AH0_actual_conc):
        """
        Calculate Oxidation Induction Time (OIT) from AH profile.

        Args:
            AH_profile (numpy.ndarray): Array of current antioxidant (AH) concentrations
                                        at different spatial points (or over time at one point).
            AH0_actual_conc (float): The initial antioxidant (AH) concentration that
                                     AH_profile should be compared against for this specific
                                     simulation run or material.

        Returns:
            numpy.ndarray: Array of OIT values (min) corresponding to AH_profile.
        """
        # Using Eq (1) from Part II [2], relative to the *initial* AH concentration used
        # OIT(t) = ti0 * ([AH](t) / [AH]0)
        # self.ti0_oit should be set appropriately for the material being simulated
        # (e.g., 165 min for Sample A in Paper [2])

        # Protect against division by zero if AH0_actual_conc is zero or negative
        if AH0_actual_conc <= 1e-12: # Using a small threshold instead of direct zero check
            # If initial AH is effectively zero, OIT is also zero
            return np.zeros_like(AH_profile) 

        # Calculate the ratio, ensuring AH_profile values are not negative
        # (though they shouldn't be if the ODEs are well-behaved)
        ah_ratio = AH_profile / AH0_actual_conc
        
        oit = self.ti0_oit * ah_ratio
        
        # OIT cannot be negative, and practically, should not exceed initial OIT significantly
        # if AH_profile cannot exceed AH0_actual_conc.
        # Clamping OIT values to be non-negative.
        #oit = np.maximum(0, oit)
        
        # Optional: If you want to strictly ensure OIT doesn't exceed ti0_oit
        # (e.g., if AH_profile due to some numerical artifact slightly exceeds AH0_actual_conc)
        # oit = np.minimum(oit, self.ti0_oit) # This would cap OIT at its initial value

        return oit

In [55]:
if __name__ == "__main__":
    pipe_thickness_m = 0.4e-3
    num_grid_points = 50     
    
    model_film = DegradationModel( 
        L=pipe_thickness_m, 
        nz=num_grid_points,
        simulation_mode='film'
    )

    model_film.ti0_oit = 291.07  #initial OIT for PE100

    ah_conc_target_for_film = (0.001 * (model_film.dens0 * 1000) / 1178 * model_film.n_AH) 
    ah0_multiplier = ah_conc_target_for_film / model_film.AH0_conc

    temp_aging_celsius = 40.0 # SUEZ experimental temperature

## === CONCENTRATION INITIALE DE DOC ===
    # Pour simulation H20
    doc_ppm_H20 = 0
    # Pour simulation HOCL
    doc_ppm_HOCL = 0.05
    # Pour test extreme
    doc_ppm_test_extreme = 10

## === DONNÉES EXPÉRIMENTALES SUEZ (H2O) ===
    experimental_times_months_H2O = np.array([0, 1, 2, 3, 4, 9]) 
    time_under_azote = 20
    experimental_oit_H2O = np.array([
        291.07 - time_under_azote, 287.95 - time_under_azote, 278.7 - time_under_azote, 
        273 - time_under_azote, 268.03 - time_under_azote, 249.3 - time_under_azote
    ])

## === DONNÉES EXPÉRIMENTALES SUEZ (HOCL) ===
    experimental_times_months_HOCL = np.array([0, 1, 2, 3, 4, 6, 9]) 
    experimental_oit_hocl = np.array([
        291.07-time_under_azote, 146.81-time_under_azote, 81.44-time_under_azote,
        52.29-time_under_azote, 43.82-time_under_azote, 39.06-time_under_azote, 29.07-time_under_azote])
    
## === DONNÉES EXPÉRIMENTALES TEST EXTREME ===
    experimental_times_months_extreme = np.array([0, 1, 2, 3, 4, 6, 9])
    experimental_oit_extreme = np.array([291.07, 0, 0, 0, 0, 0, 0]) # Hypothetical values

### === SÉLECTION DES DONNÉES EXPÉRIMENTALES EN FONCTION DU SCÉNARIO ===
    for doc_ppm_test in [doc_ppm_H20, doc_ppm_HOCL, doc_ppm_test_extreme]:
        if doc_ppm_test == doc_ppm_H20:
            experimental_times_months = experimental_times_months_H2O
            experimental_oit = experimental_oit_H2O
            if len(experimental_times_months) != len(experimental_oit):
                raise ValueError("Les tableaux de temps et d'OIT H20 expérimentaux doivent avoir la même longueur!")
            scenario_desc = "Vieillissement en H2O", doc_ppm_test 
        if doc_ppm_test == doc_ppm_HOCL:
            experimental_times_months = experimental_times_months_HOCL
            experimental_oit = experimental_oit_hocl
            if len(experimental_times_months) != len(experimental_oit):
                raise ValueError("Les tableaux de temps et d'OIT HOCl expérimentaux doivent avoir la même longueur!")
            scenario_desc = "Vieillissement en HOCl", doc_ppm_test
        if doc_ppm_test == doc_ppm_test_extreme:
            experimental_times_months = experimental_times_months_extreme
            experimental_oit = experimental_oit_extreme
            if len(experimental_times_months) != len(experimental_oit):
                raise ValueError("Les tableaux de temps et d'OIT du test extrême doivent avoir la même longueur!")
            scenario_desc = "Test extrême", doc_ppm_test

        print(f"\n=== Simulation pour {scenario_desc} ===")

### === SIMULATION ===
    
        simulated_times_years = experimental_times_months / 12.0
        simulated_oit_avg = []

        trial_methods = [
            {'method': 'Radau', 'rtol': 1e-6, 'atol': 1e-9, 'desc': 'Default Radau'},
            {'method': 'BDF',   'rtol': 1e-6, 'atol': 1e-9, 'desc': 'Default BDF'},
            {'method': 'Radau', 'rtol': 1e-3, 'atol': 1e-6, 'desc': 'Relaxed 1 Radau'},
            {'method': 'BDF',   'rtol': 1e-3, 'atol': 1e-6, 'desc': 'Relaxed 1 BDF'},]

        for trial_params in trial_methods:
            print(f"Attempting with: method={trial_params['method']}, rtol={trial_params['rtol']}, atol={trial_params['atol']} ({trial_params['desc']})")
                
            solution = model_film.simulate(
                T_celsius=temp_aging_celsius,
                DOC_ppm=doc_ppm_test,
                t_end_years=simulated_times_years[-1],
                n_timepoints=200,
                AH0_mult=ah0_multiplier,
                method=trial_params['method'], 
                rtol=trial_params['rtol'],    
                atol=trial_params['atol']
            )

            if solution.success:
                break
        print(f"    Success with method={trial_params['method']}, rtol={trial_params['rtol']}, atol={trial_params['atol']}")
        '''        
        y_data = solution.y.reshape((model_film.n_species, model_film.nz, -1))
        simulation_times_sec = solution.t
        AH0_actual_conc_used = solution.sim_params['AH0_conc_used']

        results = []
        '''

        y_data = solution.y.reshape((model_film.n_species, model_film.nz, -1))
        simulation_times_sec = solution.t
        AH0_actual_conc_used = solution.sim_params['AH0_conc_used']
## === SIMULATION SANS CORRECTION===
        print("\n--- OIT Results (Sans Correction) ---")
        results_sans = []
        for target_month in experimental_times_months:
            target_time_sec = target_month * (365.25 / 12) * 24 * 3600
            time_index = np.argmin(np.abs(simulation_times_sec - target_time_sec))
            AH_profile = y_data[model_film.idx['AH'], :, time_index]

            oit_profile = model_film.calculate_oit_sans_correction(AH_profile, AH0_actual_conc_used)
            oit_exp = experimental_oit[np.where(experimental_times_months == target_month)[0][0]]
            results_sans.append({ "Month": target_month, "OIT Avg": np.mean(oit_profile), "OIT Exp": oit_exp })
            
        oit_df_sans = pd.DataFrame(results_sans)
        print(oit_df_sans.to_string(index=False))

## === SIMULATION AVEC CORRECTION===
        print("\n--- OIT Results (Avec Correction) ---")
        results_avec = []

        for target_month in experimental_times_months:
            target_time_sec = target_month * (365.25 / 12) * 24 * 3600
            time_index = np.argmin(np.abs(simulation_times_sec - target_time_sec))
            AH_profile = y_data[model_film.idx['AH'], :, time_index]

            oit_profile = model_film.calculate_oit(AH_profile, AH0_actual_conc_used)
            oit_exp = experimental_oit[np.where(experimental_times_months == target_month)[0][0]]
            results_avec.append({ "Month": target_month, "OIT Avg": np.mean(oit_profile), "OIT Exp": oit_exp })
            
        oit_df_avec = pd.DataFrame(results_avec)
        print(oit_df_avec.to_string(index=False))

Model initialized in 'film' mode with L=4.00e-04m, nz=50.
  Film mode: beta_coeffs['betaL'] parameters set to beta_coeffs['beta0'] (Ea=0, A=1.9e-09)

=== Simulation pour ('Vieillissement en H2O', 0) ===
Attempting with: method=Radau, rtol=1e-06, atol=1e-09 (Default Radau)
  FILM MODE: Initial C0[DOC, L] set to 0.00e+00
Running simulation: T=40.0°C, DOC=0 ppm, Time=0.750 years, Mode='film'
  Solver: Radau, rtol=1.0e-06, atol=1.0e-09
Simulation duration: 6.02 seconds
Simulation successful.
    Success with method=Radau, rtol=1e-06, atol=1e-09

--- OIT Results (Sans Correction) ---
 Month    OIT Avg  OIT Exp
     0 289.614650   271.07
     1 267.065069   267.95
     2 258.346289   258.70
     3 253.285570   253.00
     4 249.935663   248.03
     9 239.363018   229.30

--- OIT Results (Avec Correction) ---
 Month    OIT Avg  OIT Exp
     0 289.614650   271.07
     1 267.065069   267.95
     2 258.346289   258.70
     3 253.285570   253.00
     4 249.935663   248.03
     9 239.363018   229.